In this notebook, we are going to load the CSV exported from the Google Doc into a `pandas` dataframe, then use the URL to scrape the HTML for the data we want, and then add that data to the dataframe. At the end, we will save the dataframe back to the CSV.

Understanding what the `requests` library returns was made easier thanks to this [post][].

[post]: http://www.compjour.org/tutorials/intro-to-python-requests-and-json/

In [1]:
# =-=-=-=-=-=-=-=-=-=-=
# Import libraries
# =-=-=-=-=-=-=-=-=-=-= 
import pandas, re, requests, csv, os
from bs4 import BeautifulSoup

In [7]:
# =-=-=-=-=-=-=-=-=-=-=
# Define functions
# =-=-=-=-=-=-=-=-=-=-= 

def parse(thesoup):
    for tag in thesoup.find_all("meta"):
        if tag.get("name", None) == "author":
            speaker = tag.get("content", None)
        if tag.get("itemprop", None) == "duration":
            length = tag.get("content", None)
        if tag.get("itemprop", None) == "uploadDate":
            published = tag.get("content", None)
        if tag.get("itemprop", None) == "interactionCount":
            views = tag.get("content", None)
        if tag.get("itemprop", None) == "description":
            description = tag.get("content", None)
    strung = ''.join([div.text for div in 
            thesoup.findAll("div", {"class": "Grid__cell flx-s:1 p-r:4"})])
    text   = re.sub(r"[\t]", "", strung).replace("\n", " ")
    return speaker, length, published, views, description, text

def to_csv(csv_file, out):
    # open file to write to.
    with open(out, "w") as out:
        # create csv.writer. 
        wr = csv.writer(out)
        # write our headers.
        wr.writerow(["speaker", "length", "published", "views", "description", "text"])
        # get all our html files.
        with open(csv_file) as f:
            colnames = f.readline().strip().split(",")
            df = pandas.read_csv(csv_file, names=colnames)
            # Create a tuple from the DF 
            rows = df.itertuples()
            # Skip the first (header) row
            next(rows) # skip the row header
            # Here's where all the lifting occurs
            for row in rows:
                # Append transcript to get the correct HTML
                the_url = row.public_url + "/transcript"
                # Use the requests library to get the data
                the_html = requests.get(the_url)
                # We're skipping the JSON niftiness of the requests response 
                # and just converting the response to text that we can parse
                the_soup = BeautifulSoup(the_html.text, "html5lib")
                # This is our parse function above that returns a 6-part tuple
                # parse the file are write the data to a row.
                wr.writerow(parse(the_soup))

In [6]:
# =-=-=-=-=-=-=-=-=-=-=
# Write the data to CSV 
# =-=-=-=-=-=-=-=-=-=-= 

to_csv("./TEDtalks_to_add.csv","added.csv")

UnboundLocalError: local variable 'speaker' referenced before assignment